# Amazon SageMaker Workshop
### _**Pipelines**_

---
In this part of the workshop we will all our previous work from the labs and will automate the whole ML workflow. With that we can make the whole process more robust and any updates to the data preparation, modeling, evaluation, inference and monitoring will be put into production faster and more reliable.

---

## Contents

1. [Background](#background) - Getting the work from previous labs.
2. [Create the training pipeline](#Create_pipeline)
    * [SageMaker Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines.html)
3. [Exercise](#a_Exercise) - adding steps to the pipeline
4. [Create the end-to-end solution automatically](#SM_Projects)
    * [SageMaker Project](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects.html) - Create end-to-end ML solutions with CI/CD by using SageMaker projects.
    * customize the project with our pipeline and code

---
<a id='background'></a>

## Background

In the previous labs we created multiple resources to prepare the data (_2-DataPrep_), train the model (_3-Modeling_), evaluate model performance (_4-Evaluation_), deploy and customize inference logic (_4-Deployment/RealTime_) and monitor the deployed model (_5-Monitoring_).

Now it's time to **bring everything together**!

We will create a pipeline with 5 steps:

1. Data preparation
2. Training
3. Evaluation
4. Approve model
5. Save to model registry step

We will build our pipeline iterating little by little.

---

### - if you _skipped_ some/all of the previous labs, follow instructions:

   - **run this [notebook](./config/pre_setup.ipynb)**

---
Load all variables (and modules) for this lab:

In [ ]:
%store -r bucket
%store -r prefix
%store -r region
%store -r docker_image_name

In [ ]:
bucket, prefix, region, docker_image_name

In [ ]:
#Supress default INFO logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [ ]:
import sagemaker 
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.session.Session()

---

# 1. Create data preparation step

Get the raw data location and the S3 URI where our code for data preparation was stored:

In [ ]:
%store -r s3uri_raw
%store -r s3_dataprep_code_uri
s3uri_raw, s3_dataprep_code_uri

In [ ]:
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
)
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)

This first step will receive some inputs:

In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

In [ ]:
# Parameters for data preparation step
input_data = ParameterString(
    name="InputDataUrl",
    default_value=s3uri_raw # S3 URI where we stored the raw data
)
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount", default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.m5.xlarge"
)

In [ ]:
from my_labs_solutions.dataprep_solution import get_dataprep_processor
sklearn_processor = get_dataprep_processor(processing_instance_type, processing_instance_count)
sklearn_processor

In [ ]:
# Processing step for feature engineering
step_process = ProcessingStep(
    name="CustomerChurnProcess",  # choose any name
    processor=sklearn_processor,
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(
            output_name="validation", source="/opt/ml/processing/validation"
        ),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code=s3_dataprep_code_uri,
    job_arguments=["--input-data", input_data],
)

## Create the first iteration of the Pipeline

We will create a simple pipeline that receives some inputs and just have 1 data preparation step:

In [ ]:
from time import strftime, gmtime
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig
from sagemaker.workflow.step_collections import RegisterModel

You can associate SageMaker Experiments with Pipelines to help track multiple moving pieces (ML hyperparameters, data, artifacts, plots, metrics, etc. - a.k.a. [ML lineage tracking](https://docs.aws.amazon.com/sagemaker/latest/dg/lineage-tracking.html)) 

In [ ]:
# Experiment configs
create_date = lambda: strftime("%Y-%m-%d-%H-%M-%S", gmtime())

experiment_name=f"pipeline-customer-churn-prediction-xgboost-{create_date()}"
trial_name=f"pipeline-framework-trial-{create_date()}"
pipeline_name = "ChurnMLPipeline"

In [ ]:
pipeline_experiment_config = PipelineExperimentConfig(
    experiment_name = experiment_name,
    trial_name = trial_name
)

In [ ]:
# Pipeline with just input parameters and 1 step for data prep
pipeline = Pipeline(
        name=pipeline_name,
        parameters=[
            input_data,
            processing_instance_type,
            processing_instance_count,
        ],
        steps=[step_process],
        sagemaker_session=sagemaker_session,
    )

In [ ]:
# Validate that pipeline was configured correctly and load its definition
import json
json.loads(pipeline.definition())

#### Ok, looks good. Let's create the pipeline:

In [ ]:
pipeline.upsert(role_arn=role)

1. Go to the pipeline and see its DAG:

<img src="./media/sm-pipeline.png" width="50%">

2. Right-click the ChurnMLPipeline -> `Open pipeline details`.

Check its DAG (with just the data prep step:

<img src="./media/sm-pipe-iter-1.png" width="50%">

3. Click on `Parameters` to see the default parameter inputs for a execution:

<img src="./media/sm-pipe-iter-1-params.png" width="100%">


> Remember that we set the inputs as:
```python
# Parameters for data preparation step
input_data = ParameterString(
    name="InputDataUrl",
    default_value=s3uri_raw # S3 URI where we stored the raw data
)
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount", default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.m5.xlarge"
)
```

**Let's programatically execute the pipeline with defaults:**


In [ ]:
execution = pipeline.start()

In [ ]:
execution.describe()

In [ ]:
execution.list_steps()

In [ ]:
# If we wanted to wait for execution to end:
# execution.wait()


4. Right-click the `Executions` tab:

<img src="./media/sm-pipe-iter-1-exec-list.png" width="100%">

5. Select the only execution (should be in status "Executing") and double click on it:

<img src="./media/sm-pipe-iter-1-exec.png" width="60%">

6. Wait for a few minutes (for the data preparation step and the SageMaker Processing Job under the hood to finish):

<img src="./media/sm-pipe-iter-1-exec-succ.png" width="60%">

7. If you go to `Experiments and trials` tab you will see that SageMaker Pipelines created an experiment called `churnmlpipeline`.

Also if we select our data prep Processing job, we can see that it correctly created 3 dataset as output: `train`, `validation` and `test`:

<img src="./media/sm-pipe-iter-1-exec-outs.png" width="100%">


---

# 2. Create modeling step

In [ ]:
%store -r s3_modeling_code_uri
%store -r train_script_name
s3_modeling_code_uri, train_script_name

In [ ]:
from my_labs_solutions.modeling_solution import get_modeling_estimator

xgb_train = get_modeling_estimator(bucket,
                                   prefix,
                                   s3_modeling_code_uri, 
                                   docker_image_name,
                                   entry_point_script = train_script_name)
xgb_train

In [ ]:
from sagemaker.inputs import TrainingInput

In [ ]:
step_train = TrainingStep(
    name="CustomerChurnTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
                    s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                        "train"
                    ].S3Output.S3Uri,
                    content_type="text/csv"
                 ),
        "validation": TrainingInput(
                    s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                        "validation"
                    ].S3Output.S3Uri,
                    content_type="text/csv"
                 )
    }
)

Notice that we can link one step's output to other steps input by accessing the properties:
```python
# Get output from processing step with key `train`
step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri
```

## Create the second iteration of the Pipeline (updating the definition)

We will update the pipeline adding an input parameter for the training Step and also the training Step itself, resulting in a pipeline with 2 step:

In [ ]:
# Add an input parameter to define the training instance type
training_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.m5.xlarge"
)

In [ ]:
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        input_data,
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
    ],
    steps=[step_process, step_train],
    sagemaker_session=sagemaker_session,
)

In [ ]:
# Update the pipeline
pipeline.upsert(role_arn=role)

1. If we go to the pipeline and click on the refresh button, we see now its 2 steps and the new input parameter:

<img src="./media/sm-pipe-iter-2.png" width="70%">

<img src="./media/sm-pipe-iter-2-params.png" width="100%">

2. Now, let's execute the new pipeline in the Studio UI. Click on `Start an execution`:

<img src="./media/sm-pipe-iter-2-exec.png" width="80%">

3. The default input configurations should appear in the Studio UI. Click on `Start`:

<img src="./media/sm-pipe-iter-2-exec-man.png" width="80%">

4. Refreshing the executions we should see:

<img src="./media/sm-pipe-iter-2-exec-man2.png" width="80%">

5. Click on `View details` or select the execution in the list (status "Executing") and double click:

<img src="./media/sm-pipe-iter-2-exec-man3.png" width="80%">

6. Wait a few minutes to the data prep Processing job and the training job finish. You should see this:

<img src="./media/sm-pipe-iter-2-exec-succ.png" width="80%">

If you click on the training step and select `Outputs` you will also be able to see the final training and validation log losses.

---

## Create evaluation step

In [ ]:
from my_labs_solutions.evaluation_solution import get_evaluation_processor
script_eval = get_evaluation_processor(docker_image_name)
script_eval

In [ ]:
from sagemaker.workflow.properties import PropertyFile

In [ ]:
evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

In [ ]:
%store -r s3url_test
%store -r s3_evaluation_code_uri
s3url_test, s3_evaluation_code_uri

In [ ]:
# Processing step for evaluation
step_eval = ProcessingStep(
        name="CustomerChurnEval",
        processor=script_eval,
        inputs=[
            ProcessingInput(
                source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
                destination="/opt/ml/processing/model",
            ),
            ProcessingInput(
                source=step_process.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test",
            ),
        ],
        outputs=[
            ProcessingOutput(
                output_name="evaluation", source="/opt/ml/processing/evaluation"
            ),
        ],
        code=os.path.join(BASE_DIR, "evaluate.py"),
        property_files=[evaluation_report],
    )

Now, notice that we get the model from the training step and also the test dataset from the data preparation step:
```python
# Get output model artifact from training step
step_train.properties.ModelArtifacts.S3ModelArtifacts

# Get the test dataset - the output of data preparation step with key `test`
step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri
```

## Create the third iteration of the Pipeline (updating the definition)

We will update the pipeline adding the evaluation step, resulting in a pipeline with 3 step: data prep, training and evaluation.

In [ ]:
pipeline = Pipeline(
        name="test",
        steps=[step_train, step_eval]
    )

In [ ]:
pipeline.upsert(role_arn=role)

---

## Create approve model and save to model registry steps

In [ ]:
from sagemaker.workflow.conditions import (
    ConditionGreaterThanOrEqualTo,
)
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.step_collections import RegisterModel

#### Create pipeline

In [ ]:
from time import strftime, gmtime
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig
from sagemaker.workflow.step_collections import RegisterModel

In [ ]:
# Experiment configs
create_date = lambda: strftime("%Y-%m-%d-%H-%M-%S", gmtime())

experiment_name=f"pipeline-customer-churn-prediction-xgboost-{create_date()}"
trial_name=f"pipeline-framework-trial-{create_date()}"

In [ ]:
pipeline_experiment_config = PipelineExperimentConfig(
    experiment_name = experiment_name,
    trial_name = trial_name
)

In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

In [ ]:
training_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.m5.xlarge"
)
model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval",  # ModelApprovalStatus can be set to a default of "Approved" if you don't want manual approval.
)
input_data = ParameterString(
    name="InputDataUrl",
    default_value=f"s3://EXAMPLE-BUCKET/PATH/TO/RawData.csv",  # Change this to point to the s3 location of your raw input data.
)

In [ ]:
pipeline = Pipeline(
        name=pipeline_name,
        parameters=[
            processing_instance_type,
            processing_instance_count,
            training_instance_type,
            model_approval_status,
            input_data,
        ],
        steps=[step_process, step_train, step_eval, step_cond],
        sagemaker_session=sagemaker_session,
    )

In [ ]:
import json

Visualize pipeline definition

In [ ]:
json.loads(pipeline.definition())

In [ ]:
pipeline.upsert(role_arn=role)

Execute with defaults:

In [ ]:
execution = pipeline.start()

In [ ]:
execution.describe()

In [ ]:
execution.list_steps()

In [ ]:
execution.wait()

Read evaluation report

In [ ]:
evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
    step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
))
json.loads(evaluation_json)


Run another time with other parameters:

In [ ]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.stop()

In [ ]:
pipeline.delete()

In [ ]:
# get estimators and processors from previous labs
# add other metrics in eval and also graphics
# send hyperparamenters to Pipeline as parameter (I believe it's possible sending in InputConfig)
# Pipeline load definition?

<a id='SM_Projects'></a>
# Customizing the Build/Train/Deploy MLOps Project Template

We recently announced [Amazon SageMaker Pipelines](https://aws.amazon.com/sagemaker/pipelines/), the first 
purpose-built, easy-to-use Continuous Integration and Continuous Delivery (CI/CD) service for machine learning. 
SageMaker Pipelines has three main components which improves the operational resilience and reproducibility of your 
workflows: Pipelines, Model Registry, and Projects. 

SageMaker Projects introduce MLOps templates that automatically provision the underlying resources needed to enable 
CI/CD capabilities for your Machine Learning Development Lifecycle (MLDC). Customers can use a number of built-in 
templates or create your own custom templates.

This example will focus on using one of the MLOps templates to bootstrap your ML project and establish a CI/CD 
pattern from seed code. We’ll show how to use the built-in Build/Train/Deploy Project template as a base for a 
customer churn classification example. This base template will enable CI/CD for training machine learning models, 
registering model artifacts to the Model Registry, and automating model deployment with manual approval and automated 
testing.

## MLOps Template for Build, Train, and Deploy

We’ll start by taking a detailed look at what AWS services are launched when this build, train, deploy MLOps template 
is launched. Later, we’ll discuss how the skeleton can be modified for a custom use case. 

To get started with SageMaker Projects, [they must be first enabled in the SageMaker Studio console](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-studio-updates.html). 
This can be done for existing users or while creating new ones:

<img src="media/enable_projects.png">

Within Amazon SageMaker Studio, you can now select “Projects” from a drop-down menu on the “Components and registries” 
tab as shown below:

<img src="media/select_projects.png">

From the projects page you’ll have the option to launch a pre-configured SageMaker MLOps template. We'll select the build, train and deploy template:

<img src="media/create_project.png">

NOTE: Launching this template will kick off a model building pipeline by default and will train a regression model. This will incur a small cost.

Once the project is created from the MLOps template, the following architecture will be deployed:

<img src="media/deep_dive.png">


## Modifying the Seed Code for Custom Use Case

After your project has been created the architecture shown above will be deployed and the visualization of the 
Pipeline will be available in the “Pipelines” drop down menu within SageMaker Studio.

In order to modify the seed code from this launched template, we’ll first need to clone the AWS CodeCommit 
repositories to our local SageMaker Studio instance. From the list of projects, select the one that was just 
created. Under the “Repositories” tab you can select the hyperlinks to locally clone the AWS CodeCommit repos:

<img src="media/clone_repos.png">


### ModelBuild Repo

The SageMaker project template will create this repositories.

In the `...-modelbuild` repository there's the code for preprocessing, training, and evaluating the model. 
The seed code trains and evaluates a model on the [UCI Abalone dataset](https://archive.ics.uci.edu/ml/datasets/abalone):

<img src="media/repo_directory.png">


**In our case we want to create a pipeline for predicting Churn (part 1 of the lab).** We can modify these files in order to solve our own customer churn use-case.


We’ll need a dataset accessible to the project (_Churn dataset_). 

The easiest way to do this is run the following in our notebook:

**Ok, now we have donwloaded the Churn dataset and uploaded it to our S3 Bucket that is accessible to the SageMaker Project role.**

---

### Modifying the code for the Churn problem

This is the sample structure of the Project (Abalone):

<img src="media/repo_directory.png">


#### We'll need to:
1. rename the `abalone` directory to `customer_churn`
2. replace `codebuild-buildspec.yml` in your current Studio project (Abalone) with the one found in [modelbuild/codebuild-buildspec.yml](modelbuild/codebuild-buildspec.yml) (Churn)
3. replace the `preprocess.py`, `evaluate.py` (of the sample Abalone) with the ones found in `modelbuild/pipelines/customer_churn`
4. replace `pipeline.py`(Abalone) with the one found in `modelbuild/pipelines/customer_churn/pipeline.py`

    
5. **In the `pipeline.py` file you'll need to replace the `default_value` of `InputDataURL` with the URL you obtained when uploading the data above.**
    
```python
#in pipeline.py
...
input_data = ParameterString(
    name="InputDataUrl",
    default_value=f"s3://EXAMPLE-BUCKET/PATH/TO/RawData.csv",  # Change this to point to the s3 location of your raw input data.
)
...
```


## Trigger a new training Pipeline Execution through git commit

By committing these changes to the AWS CodeCommit repository (easily done in SageMaker Studio source control tab), a 
new Pipeline execution will be triggered since there is an EventBridge monitoring for commits.  After a few moments, 
we can monitor the execution by selecting your Pipeline inside of the SageMaker Project.

<img src="media/git_push.png">

This triggers the pipelines for training. Go to our `“Pipelines”` tab inside of the SageMaker Project. Click on our only pipeline. And you'll see:

<img src="media/execute_pipeline.png">

Select the most recent execution:

<img src="media/dag.png">


## Trigger the ModelDeploy Pipeline

Once the train pipeline is completed, we can go to our `“Model groups”` tab inside of the SageMaker Project and inspect the metadata attached to the model artifacts. If everything looks good, we can manually approve the model:

<img src="media/model_metrics.png">

<img src="media/approve_model.png">

This approval will trigger the ModelDeploy pipeline (in CodePipeline):

<img src="media/execute_pipeline_deploy.png">

After we deploy to a staging environment and run some tests, we will have to **approve the deployment to production** by approving in the `ApproveDeployment` stage:

<img src="media/approve_deploy_prod.png">



Finally, if we go back to Studio, we will see the Production endpoint for real time inference.

<img src="media/endpoints.png">

---